In [ ]:
import numpy as np
from itertools import combinations, filterfalse
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import random
import sys
import os
import pickle
import gc


In [ ]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 3.5MB/s 
     |████████████████████████████████| 3.7MB 17.3MB/s 
     |████████████████████████████████| 1.1MB 43.2MB/s 
     |████████████████████████████████| 890kB 43.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=31a07a2a306c76240382ba7c1ef7f57d8cb2d67b9353e3293cd1b6c2c16f4479
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Config

In [ ]:
import torch
import transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = 'large' #'base'
nlayer = 12 if config == 'base' else 24
nsamples = 5000

model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased', output_hidden_states=True).to(device)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-'+config+'-uncased')
# turn on eval mode
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

# Analogies

In [ ]:
from torch.utils.data import DataLoader, Dataset
import tqdm

class GoogleAnalogies:
    def __init__(self, path='/content/drive/My Drive/questions-words.txt'):
        self.Lset = set()
        self.Rset = set()
        # this time only want the words in vocab !!!!!!!!!!
        with open(path,'r',encoding='utf8') as f:
            for line in f.readlines():
                line = line.strip().split(' ')
                self.Rset.update([line[1],line[3]])
                for w in [line[0],line[2]]:
                    if w in tokenizer.vocab.keys():
                        self.Lset.add(w)
                for w in [line[1],line[3]]:
                    if w in tokenizer.vocab.keys():
                        self.Rset.add(w)
        self.Lset = tokenizer.convert_tokens_to_ids(list(self.Lset))
        self.Rset = tokenizer.convert_tokens_to_ids(list(self.Rset))
        self.pieces = self.Lset+self.Rset
             
    def get_embeddings_from_cropus(self, corpus, layer=11):
        #balanced
        L = torch.Tensor().to(device)
        R = torch.Tensor().to(device)
        for i in tqdm.trange(10*nsamples):# just to get a larger range
            # get useful idxs
            vecs = model(corpus.__getitem__(i))[1][layer+1][0][1:-1]#!!!!!!!!!!!
            if L.shape[0]<nsamples//2:
                for j in range(vecs.shape[0]):
                    if corpus.labels[i][j]==1:
                        L = torch.cat((L, vecs[j].unsqueeze(0)))
            else:
                L = L[:nsamples//2]
            if R.shape[0]<nsamples//2:
                for j in range(vecs.shape[0]):
                    if corpus.labels[i][j]==-1:
                        R = torch.cat((R, vecs[j].unsqueeze(0)))
            else:
                R = R[:nsamples//2]
            if L.shape[0]+R.shape[0]>=nsamples:
                break
        return L.cpu(), R.cpu()
    
    def get_LR_from_cropus(self, corpus, LR='L', layer=11):
        #balanced
        flag = 1 if LR== 'L' else -1
        L = torch.Tensor().to(device)
        #for i in tqdm.trange(nsamples):
        iterator = tqdm.trange(corpus.__len__())
        for i in iterator:
            # get useful idxs
            with torch.no_grad():
                vecs = model(corpus.__getitem__(i))[1][layer+1][0][1:-1]#!!!!!!!!!!!
            if L.shape[0]<nsamples//2:
                for j in range(vecs.shape[0]):
                    if corpus.labels[i][j]==flag:
                        L = torch.cat((L, vecs[j].unsqueeze(0)))
            else:
                L = L[:nsamples//2]
            if L.shape[0]>=nsamples//2:
                iterator.close()
                break
        return L.cpu()

    

In [ ]:
analogy = GoogleAnalogies('/content/drive/My Drive/questions-words.txt')

# Dataset

In [ ]:
from torch.utils.data import DataLoader, Dataset
import tqdm
import pickle

class TextDataset(Dataset):
    def __init__(self, path="/content/drive/My Drive/wikien_senttok_500k.txt"):
        self.examples, self.labels = pickle.load(open(path, 'rb'))
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, i):
        return tokenizer.prepare_for_model(self.examples[i][:510], return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
        #return torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)

wiki = TextDataset("/content/drive/My Drive/wikien_senttok_500k.txt")

# DensRay

In [ ]:
import tqdm

class DensRay:
    def __init__(self, Lemb, Remb):
        self.lemb = Lemb
        self.remb = Remb

    def fit(self, weights=None, normalize_D=True):
        """Fit DensRay
        Args:
            weights: only for binary model; how to weight the two
                summands; if none
                
                
                : apply dynamic weighting. Example input: [1.0, 1.0]
            normalize_D: bool whether to normalize the difference vectors with l2 norm
        """
        self.computeA_binary_part1(normalize_D=normalize_D)
        #self.A_equal = self.opsum(self.lemb) + self.opsum(self.remb)
        #self.A_unequal = self.opsum(self.lemb, self.remb) + self.opsum(self.remb, self.lemb)
        self.computeA_binary_part2(weights=weights)
        self.compute_trafo()
        #self.compute_mean_var()

    @staticmethod
    def opsum(a, b=None):
        if b is None: b = a
        out = -torch.ger(a.sum(dim=0), b.sum(dim=0))
        out = out + out.T
        out += b.shape[0] * torch.mm(a.T,a)
        out += a.shape[0] * torch.mm(b.T,b)
        return out

    @staticmethod
    def outer_product_sub_binary(v, M, normD):
        """Helper function to compute the sum of outer products

        While it is not very readable, it is more efficient than
        a brute force implementation.
        """
        d = v.unsqueeze(0) - M
        if normD:
            norm = d.norm(dim=1)
            norm[norm == 0] = 1
            d = d / (norm.unsqueeze(0).T)
        return torch.mm(d.T, d)
    
    def computeA_binary_part1(self, normalize_D=False):
        """First part of computing the matrix A.
        Args:
            normalize_D: bool whether to normalize the difference vectors with l2 norm.
        """
        dim = self.lemb.shape[1]
        self.A_equal = torch.zeros((dim, dim)).to(device)
        self.A_unequal = torch.zeros((dim, dim)).to(device)
        for ipos in tqdm.trange(self.lemb.shape[0]):
            v = self.lemb[ipos]
            self.A_equal += self.outer_product_sub_binary(v, self.lemb, normalize_D)
            self.A_unequal += self.outer_product_sub_binary(v, self.remb, normalize_D)
        for ineg in tqdm.trange(self.remb.shape[0]):
            v = self.remb[ineg]
            self.A_equal += self.outer_product_sub_binary(v, self.remb, normalize_D)
            self.A_unequal += self.outer_product_sub_binary(v, self.lemb, normalize_D)

    def computeA_binary_part2(self, weights=None):
        """Second part of computing the matrix A.
        Args:
            weights: only for binary model; how to weight the two 
                summands; if none: apply dynamic weighting. Example input: [1.0, 1.0]
        """
        if weights is None:
            weights = [1 / (2 * self.lemb.shape[0] * self.remb.shape[0]), 1 /
                       (self.lemb.shape[0]**2 + self.remb.shape[0]**2)]
        # normalize matrices for numerical reasons
        # note that this does not change the eigenvectors
        n1 = self.A_unequal.max()
        n2 = self.A_equal.max()
        weights = [weights[0] / max(n1, n2), weights[1] / max(n1, n2)]
        self.A = weights[0] * self.A_unequal - weights[1] * self.A_equal

    def compute_trafo(self):
        """Given A, this function computes the actual Transformation.
        It essentially just does an eigenvector decomposition.
        """
        eigvals, eigvecs = self.A.symeig(eigenvectors=True)
        # need to sort the eigenvalues
        idx = eigvals.argsort(descending=True)
        eigvals, self.eigvecs = eigvals[idx], eigvecs[:, idx]
    
    def compute_mean_var(self):
        first_dim = torch.mm(torch.cat((self.lemb, self.remb)), self.eigvecs)[:, 0]
        self.mean = first_dim.mean()
        self.std = first_dim.var().sqrt()

# Conceptor

In [ ]:
import numpy as np

def train_Conceptor(x, alpha = 1):
    print("starting...")
    #x = orig_embd.vectors
    print(x.shape)
    #Calculate the correlation matrix
    R = x.T.dot(x)/(x.shape[0])
    #Calculate the conceptor matrix
    C = R @ (np.linalg.inv(R + alpha ** (-2) * np.eye(x.shape[1])))
    print("C calculated")
    dim = C.shape[0]
    not_C = np.eye(dim) - C
    return not_C

def process_cn_matrix(subspace, alpha = 2):
    """Returns the conceptor negation matrix
    Arguments
            subspace : n x d matrix of word vectors from a oarticular subspace
            alpha : Tunable parameter
    """
    # Compute the negation conceptor matrix
    negC = train_Conceptor(subspace, alpha)    
    return negC

## Train

In [ ]:
def save_lr(corpus=None):
    for l in range(nlayer):
        L = analogy.get_LR_from_cropus(corpus,'L', l)
        R = analogy.get_LR_from_cropus(corpus,'R', l)
        torch.save(L, '/content/drive/My Drive/L'+config+'_'+str(nsamples)+'_'+str(l)+'.pt')
        torch.save(R, '/content/drive/My Drive/R'+config+'_'+str(nsamples)+'_'+str(l)+'.pt')
        print("Layer saved: ", l)

save_lr(wiki)


  0%|          | 4/500001 [00:00<4:10:26, 33.28it/s]

Layer saved:  0


  0%|          | 4/500001 [00:00<4:16:05, 32.54it/s]

Layer saved:  1


  0%|          | 4/500001 [00:00<4:02:35, 34.35it/s]

...

Layer saved:  21


  0%|          | 4/500001 [00:00<4:06:30, 33.81it/s]

Layer saved:  22


  1%|▏         | 6652/500001 [03:16<4:03:08, 33.82it/s]

Layer saved:  23


In [ ]:
from sklearn.decomposition import PCA

def train_only(corpus=None, layer=11):
    # prepare training data for densray
    L = torch.load('/content/drive/My Drive/L'+config+'_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)
    R = torch.load('/content/drive/My Drive/R'+config+'_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)
    Z = torch.cat((L,R)).cpu().detach().numpy()
    # compute conceptor
    negC = process_cn_matrix(Z)
    torch.save(torch.Tensor(negC).cpu(), '/content/drive/My Drive/negc'+config+'_'+str(nsamples)+'_'+str(layer)+'.pt')
    # compute hard debiasing
    #pca = PCA(n_components = 1)
    #pca.fit(Z)
    #pc1 = np.array(pca.components_)
    #torch.save(torch.Tensor(pc1).cpu(), '/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(layer)+'.pt')
    # compute densray
    #densray = DensRay(L,R)
    #densray.fit(normalize_D=True)
    #torch.save(densray.eigvecs, '/content/drive/My Drive/eigvecs_'+config+'_new2_'+str(nsamples)+'_'+str(layer)+'.pt')
    print(str(layer)+' saved')


In [ ]:
for l in range(nlayer):
    print("Layer ", l)
    train_only(wiki, layer=l)

Layer  0
starting...
(5000, 1024)
C calculated
0 saved
Layer  1
starting...
(5000, 1024)
C calculated
1 saved
Layer  2
starting...
(5000, 1024)
C calculated
2 saved
Layer  3
starting...
(5000, 1024)
C calculated
3 saved
Layer  4
starting...
(5000, 1024)
C calculated
4 saved
Layer  5
starting...
(5000, 1024)
C calculated
5 saved
Layer  6
starting...
(5000, 1024)
C calculated
6 saved
Layer  7
starting...
(5000, 1024)
C calculated
7 saved
Layer  8
starting...
(5000, 1024)
C calculated
8 saved
Layer  9
starting...
(5000, 1024)
C calculated
9 saved
Layer  10
starting...
(5000, 1024)
C calculated
10 saved
Layer  11
starting...
(5000, 1024)
C calculated
11 saved
Layer  12
starting...
(5000, 1024)
C calculated
12 saved
Layer  13
starting...
(5000, 1024)
C calculated
13 saved
Layer  14
starting...
(5000, 1024)
C calculated
14 saved
Layer  15
starting...
(5000, 1024)
C calculated
15 saved
Layer  16
starting...
(5000, 1024)
C calculated
16 saved
Layer  17
starting...
(5000, 1024)
C calculated
17 

# WEAT Algorithm
The Word Embeddings Association Test (WEAT), as proposed by Calikson et. al., is a statistical test analogous to the Implicit Association Test (IAT) which helps quantify human biases in textual data. WEAT uses the cosine similarity between word embeddings which is analogous to the reaction time when subjects are asked to pair two concepts they find similar in the IAT.  WEAT considers two sets of target words and two sets of attribute words of equal size. The null hypothesis is that there is no difference between the two sets of target words and the sets of attribute words in terms of their relative similarities measured as the cosine similarity between the embeddings. For example, consider the target sets as words representing *Career* and *Family* and let the two sets of attribute words be *Male* and *Female* in that order. The null hypothesis states that *Career* and *Family* are equally similar (mathematically, in terms of the mean cosine similarity between the word representations) to each of the words in the *Male* and *Female* word lists. 

REF: https://gist.github.com/SandyRogers/e5c2e938502a75dcae25216e4fae2da5



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def swAB(W, A, B):
    """Calculates differential cosine-similarity between word vectors in W, A and W, B
        Arguments
                W, A, B : n x d matrix of word embeddings stored row wise
    """
    WA = cosine_similarity(W,A)
    WB = cosine_similarity(W,B)
    
    #Take mean along columns
    WAmean = np.mean(WA, axis = 1)
    WBmean = np.mean(WB, axis = 1)
    
    return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
    """Calculates test-statistic between the pair of association words and target words
        Arguments
                X, Y, A, B : n x d matrix of word embeddings stored row wise
        Returns
                Test Statistic
    """
    return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

## Effect Size (d-value)

The ''effect size'' is a normalized measure of how separated the two distributions are.

In [ ]:
def weat_effect_size(X, Y, A, B, embd):
    """Computes the effect size for the given list of association and target word pairs
        Arguments
                X, Y : List of association words
                A, B : List of target words
                embd : Dictonary of word-to-embedding for all words
        Returns
                Effect Size
    """
    Xmat = np.array([embd[w] for w in X if w in embd])
    Ymat = np.array([embd[w] for w in Y if w in embd])
    Amat = np.array([embd[w] for w in A if w in embd])
    Bmat = np.array([embd[w] for w in B if w in embd])
    XuY = list(set(X).union(Y))
    XuYmat = []
    for w in XuY:
        if w.lower() in embd:
            XuYmat.append(embd[w.lower()])
    XuYmat = np.array(XuYmat)
    d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))
    return d

## P-Value

The one-sided P value measures the likelihood that a random permutation of the attribute words would produce at least the observed test statistic

In [ ]:
def random_permutation(iterable, r=None):
    """Returns a random permutation for any iterable object"""
    pool = tuple(iterable)
    r = len(pool) if r is None else r
    return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample=None):
    np.random.seed(42)
    random.seed(42)
    """Computes the one-sided P value for the given list of association and target word pairs
        Arguments
                X, Y : List of association words
                A, B : List of target words
                embd : Dictonary of word-to-embedding for all words
                sample : Number of random permutations used.
        Returns
    """
    size_of_permutation = min(len(X), len(Y))
    X_Y = X + Y
    test_stats_over_permutation = []
    
    Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
    Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
    Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
    Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
    
    if not sample:
        permutations = combinations(X_Y, size_of_permutation)
    else:
        permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]
        
    for Xi in permutations:
        Yi = filterfalse(lambda w:w in Xi, X_Y)
        Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
        Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
        test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))
        
    unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
    
    is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
    
    return is_over.sum() / is_over.size

# Dataset

In [ ]:
def category(type=1):
    m_words = "male, man, boy, brother, he, him, his, son, father, uncle, grandfather".lower().replace(" ", "").split(',')
    f_words = "female, woman, girl, sister, she, her, hers, daughter, mother, aunt, grandmother".lower().replace(" ", "").split(',')
    if type == 1:
        #B1: career vs family
        x_words = "executive, management, professional, corporation, salary, office, business, career".lower().replace(" ", "").split(',')
        y_words = "home, parents, children, family, cousins, marriage, wedding, relatives".lower().replace(" ", "").split(',')
    elif type == 2:
        #B2: maths vs arts
        x_words = "math, algebra, geometry, calculus, equations, computation, numbers, addition".lower().replace(" ", "").split(',')
        y_words = "poetry, art, Shakespeare, dance, literature, novel, symphony, drama".lower().replace(" ", "").split(',')
    elif type == 3:
        #B3: science vs arts
        x_words = "science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy".lower().replace(" ", "").split(',')
        y_words = "poetry, art, Shakespeare, dance, literature, novel, symphony, drama".lower().replace(" ", "").split(',')
    elif type == 4:
        #B4: intelligence vs appearance
        x_words = "precocious, resourceful, inquisitive, genius, inventive, astute, adaptable, reflective,discerning, intuitive, inquiring, judicious, analytical, apt, venerable, imaginative,shrewd, thoughtful, wise, smart, ingenious, clever, brilliant, logical, intelligent".lower().replace(" ", "").split(',')
        y_words = "alluring, voluptuous, blushing, homely, plump, sensual, gorgeous, slim, bald,athletic, fashionable, stout, ugly, muscular, slender, feeble, handsome, healthy,attractive, fat, weak, thin, pretty, beautiful, strong".lower().replace(" ", "").split(',')
    elif type == 5:
        #B5: strength vs weakness
        x_words = "power, strong, confident, dominant, potent, command, assert, loud, bold, succeed,triumph, leader, shout, dynamic, winner".lower().replace(" ", "").split(',')
        y_words = "weak, surrender, timid, vulnerable, weakness, wispy, withdraw, yield, failure, shy,follow, lose, fragile, afraid, loser".lower().replace(" ", "").split(',')
    return m_words, f_words, x_words, y_words

def category_2(type=1):
    if type == 1:
        #Career/Family
        m_words = 'John, Paul, Mike, Kevin, Steve, Greg, Jeff, Bill'.lower().replace(" ", "").split(',')
        f_words = 'Amy, Joan, Lisa, Sarah, Diana, Kate, Ann, Donna'.lower().replace(" ", "").split(',')
        x_words = 'executive, management, professional, corporation, salary, office, business, career'.lower().replace(" ", "").split(',')
        y_words = 'home, parents, children, family, cousins, marriage, wedding, relatives'.lower().replace(" ", "").split(',')
    elif type == 2:
        #Math/Art
        m_words = 'math, algebra, geometry, calculus, equations, computation, numbers, addition'.lower().replace(" ", "").split(',')
        f_words = 'poetry, art, dance, literature, novel, symphony, drama, sculpture'.lower().replace(" ", "").split(',')
        x_words = 'male, man, boy, brother, he, him, his, son'.lower().replace(" ", "").split(',')
        y_words = 'female, woman, girl, sister, she, her, hers, daughter'.lower().replace(" ", "").split(',')
    elif type == 3:
        #Science/Art
        m_words = 'science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy'.lower().replace(" ", "").split(',')
        f_words = 'poetry, art, Shakespeare, dance, literature, novel, symphony, drama'.lower().replace(" ", "").split(',')
        x_words = 'brother, father, uncle, grandfather, son, he, his, him'.lower().replace(" ", "").split(',')
        y_words = 'sister, mother, aunt, grandmother, daughter, she, hers, her'.lower().replace(" ", "").split(',')
    return m_words, f_words, x_words, y_words

# Get Embeddings

In [ ]:
import densray_bert


def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer == -1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_noavg_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_noavg_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_noavg_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_noavg_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


def get_bert_embedding(model, wordlist, is_targets=1):
    vecss = torch.Tensor().to(device)
    for w in wordlist:
        text = w + ' is ' + tokenizer.mask_token + '.' if is_targets else tokenizer.mask_token + ' is ' + w + '.'
        vec = tokenizer.prepare_for_model(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text)),
                                            return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
        vecs = vec.clone().detach()
        # get output
        vecs = model.bert(vecs)[0]#[2][nlayer]
        vecs = vecs[0][1:-4,:].mean(dim=0).unsqueeze(0) if is_targets else vecs[0][3:-2,:].mean(dim=0).unsqueeze(0)
        vecss = torch.cat((vecss,vecs))
    return vecss

def eval_per_layer(layer=-2):
    config_class = get_eigvecs_dict(layer)
    model = densray_bert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(l)).to(device)
    # turn on eval mode
    model.eval()
    m = get_bert_embedding(model, m_words, is_targets=0).cpu().detach().numpy()
    f = get_bert_embedding(model, f_words, is_targets=0).cpu().detach().numpy()
    x = get_bert_embedding(model, x_words, is_targets=1).cpu().detach().numpy()
    y = get_bert_embedding(model, y_words, is_targets=1).cpu().detach().numpy()
    embed = {}
    for i in range(len(m_words)): embed[m_words[i]] = m[i]
    for i in range(len(f_words)): embed[f_words[i]] = f[i]
    for i in range(len(x_words)): embed[x_words[i]] = x[i]
    for i in range(len(y_words)): embed[y_words[i]] = y[i]
    return embed

ModuleNotFoundError: ignored

# Go!

In [ ]:
for t in range(1,4):
    m_words, f_words, x_words, y_words = category_2(t)
    print('d    d_densray |d|-|d_densray|   p   p-densray   p_densray-p')
    l=-2
    # no densray
    embed = eval_per_layer(layer=l)
    d =  weat_effect_size(x_words, y_words, m_words, f_words, embed)
    p = weat_p_value(x_words, y_words, m_words, f_words, embed, sample=1000)
    #densray
    for l in range(-1, 0):
        # densray
        embed = eval_per_layer(layer=l)
        d_densray =  weat_effect_size(x_words, y_words, m_words, f_words, embed)
        p_densray = weat_p_value(x_words, y_words, m_words, f_words, embed, sample=1000)
        print(round(d,4), round(d_densray,4), round(abs(d)-abs(d_densray),4), 
              round(p,4), round(p_densray,4), round(p_densray-p,4))
    print('\n')

d    d_densray |d|-|d_densray|   p   p-densray   p_densray-p


NameError: ignored

In [ ]:
eigviecs.shape

torch.Size([768, 768])

In [ ]:
pc1=torch.load('/content/drive/My Drive/pc1base_'+str(5000)+'_'+str(11)+'.pt')

In [ ]:
pc1.shape

torch.Size([1, 768])

In [ ]:
pc1

In [ ]:
import re
pc1=torch.load('/content/drive/My Drive/pc1base_'+str(5000)+'_'+str(11)+'.pt')
titles='boy girl brother sister dad mom father mother he she king queen uncle aunt nephew niece groom bride prince princess'.split(' ')
lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' .') for i in titles]
examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
baseline = []
for i in range(len(examples)):
    ei = torch.tensor(examples[i], dtype=torch.long).unsqueeze(0).to(device)
    output = model(ei)[1][11][0]#[5]
    print(titles[i],torch.mm(output,pc1.T.to(device))[4].data)

boy tensor([1.5560], device='cuda:0')
girl tensor([-1.7348], device='cuda:0')
brother tensor([2.3765], device='cuda:0')
sister tensor([-0.7137], device='cuda:0')
dad tensor([1.6269], device='cuda:0')
mom tensor([-1.8257], device='cuda:0')
father tensor([1.8576], device='cuda:0')
mother tensor([-1.9006], device='cuda:0')
he tensor([3.5507], device='cuda:0')
she tensor([-3.3792], device='cuda:0')
king tensor([0.9770], device='cuda:0')
queen tensor([-1.5296], device='cuda:0')
uncle tensor([1.8033], device='cuda:0')
aunt tensor([-1.0254], device='cuda:0')
nephew tensor([1.5656], device='cuda:0')
niece tensor([-1.1062], device='cuda:0')
groom tensor([0.5650], device='cuda:0')
bride tensor([-1.0015], device='cuda:0')
prince tensor([0.9464], device='cuda:0')
princess tensor([-1.1232], device='cuda:0')


In [ ]:
import re
eigviecs=torch.load('/content/drive/My Drive/eigvecs_base_new_'+str(5000)+'_'+str(9)+'.pt')
titles='boy girl brother sister dad mom father mother he she king queen uncle aunt nephew niece groom bride prince princess'.split(' ')
lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' .') for i in titles]
examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=128)["input_ids"]
baseline = []
for i in range(len(examples)):
    ei = torch.tensor(examples[i], dtype=torch.long).unsqueeze(0).to(device)
    output = model(ei)[1][6][0]#[5]
    print(titles[i],torch.mm(output,eigviecs.to(device))[4][0])

boy tensor(-2.8731, device='cuda:0', grad_fn=<SelectBackward>)
girl tensor(2.8983, device='cuda:0', grad_fn=<SelectBackward>)
brother tensor(-1.6107, device='cuda:0', grad_fn=<SelectBackward>)
sister tensor(4.3276, device='cuda:0', grad_fn=<SelectBackward>)
dad tensor(-1.6326, device='cuda:0', grad_fn=<SelectBackward>)
mom tensor(3.0312, device='cuda:0', grad_fn=<SelectBackward>)
father tensor(-2.0066, device='cuda:0', grad_fn=<SelectBackward>)
mother tensor(3.7363, device='cuda:0', grad_fn=<SelectBackward>)
he tensor(-4.0711, device='cuda:0', grad_fn=<SelectBackward>)
she tensor(3.0651, device='cuda:0', grad_fn=<SelectBackward>)
king tensor(-2.3243, device='cuda:0', grad_fn=<SelectBackward>)
queen tensor(3.6856, device='cuda:0', grad_fn=<SelectBackward>)
uncle tensor(-1.4492, device='cuda:0', grad_fn=<SelectBackward>)
aunt tensor(4.2484, device='cuda:0', grad_fn=<SelectBackward>)
nephew tensor(-1.5659, device='cuda:0', grad_fn=<SelectBackward>)
niece tensor(3.5433, device='cuda:0', gr

In [ ]:
output[1][12][0]#[5]

torch.Size([768])

In [ ]:
examples

[[101, 103, 2003, 1037, 2158, 1012, 102],
 [101, 103, 2003, 1037, 2450, 1012, 102],
 [101, 103, 2003, 1037, 2269, 1012, 102],
 [101, 103, 2003, 1037, 2388, 1012, 102]]